## Get data from NSE

In [1]:



import requests
import pandas as pd
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)

fromdate = datetime.strftime(datetime.today(), '%d-%m-%Y')
todate = datetime.today() - timedelta(days=120)
enddate = datetime.strftime(todate, '%d-%m-%Y')
url = 'https://www.nseindia.com/api/corporates-pit?index=equities&from_date=' + enddate + '&to_date=' + fromdate
print(url)

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Referer': 'https://www.nseindia.com',
    'X-Requested-With': 'XMLHttpRequest'
}

session = requests.Session()
session.headers.update(headers)


try:
    # Initial request to get the cookies
    st = session.get('https://www.nseindia.com', timeout=5,headers=headers)
    #st.raise_for_status()  # Check if the request was successful
    print(st.cookies.get_dict())

    # Request to fetch the data
    response = session.get(url, headers=headers, cookies=st.cookies.get_dict())
    print(response.status_code)

    if response.status_code == 200:
        data = response.json()
        print(data)
    elif response.status_code == 401:
        print("Unauthorized access. Please check your authentication credentials.")
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

https://www.nseindia.com/api/corporates-pit?index=equities&from_date=19-12-2024&to_date=18-04-2025
{'_abck': '34140C5818C0B74D448BF310DB9636EF~-1~YAAQhNcLF++pxT6WAQAA2pgIRQ2rohtpw7lYySHfw4r6c0YMYFxudbqGqCH5TWPLojnKd6B+SCrxdmUtN0HmrYhOQqZUpAI5z4CSwNsDcvhw+bQ2+YfwA3tkOYb8XA93CdKnAujOHHyXrGePfTo8TtyVdQiRLrWiiNddMsMHhBEvOzYkgHqhFBeh042spI0oOB0E5sZyZGrlDzXydRSZ4X4jDYq9QMgrZAKnEKd1s6KRlZY7+1vzW+A9Lf78kmubZ+ZEfp5NbO6+Ghttvz626+8yp6FVOnb2hBiEUpLtxbqIac8DHOlrjbtpnNG7aBmVgKrpkLzufcmqfGcJZ3jtNplfYFwE52fRp+THxWHSUyw8ktz8NRwQ/3Wa/BwSV7Pya82S1QDv0hCwBtVQfNrJ8++6AAkI4202o4qjD2TM2w==~-1~-1~-1', 'bm_sz': '7CCCC67522287577B60989D7CCBD034A~YAAQhNcLF/CpxT6WAQAA2pgIRRtnffQb4PzFwFeMzIcTHGGIdGZNp/dX6U28afBDCV5eRRkKlH1pfrfGLUYTW9FbroLA3YZ23VJmm2YWKFx4skC0hZ9t+XKdVAta3yMPE2wXV27y2IP6eDL8C6R1m0JUvq6cApIc9/6q3b0LNrWHGKJxovuASD4lS9Yc0n0p+v7p1elAPw9NNUBboSXEaF6dZ0rcFDh6o5IttHPORdQkX14r0qJKSthNZeR91TigZ2qmm0xsqpDrZLUHMxprb9xvTYIInXWK1XUkoso32g4WqFoNdo4qMsB/Njlli+ceYBaQ47JCl5Jf8cf7hG8h33stJGZo5hJMtRqHBvWh~4604215~34

## Convert Json to Dataframe

In [ ]:
df = pd.DataFrame(data['data'])
df

In [ ]:
df.replace("-", 0, inplace=True)

In [ ]:

df["secVal"] = pd.to_numeric(df["secVal"])
df['acqfromDt'] = pd.to_datetime(df['acqfromDt'],format='mixed')
df

In [ ]:

personCat = ['Promoters','Promoter Group']
df = df[(df['personCategory'].isin(personCat)) & (df['acqMode'] == 'Market Purchase') & (df['tdpTransactionType'] == 'Buy') ]
df

In [ ]:

df[df['symbol'] == "POONAWALLA"]

In [ ]:

df1 = df.groupby(['symbol']).agg({'secVal':'sum','acqfromDt':'max'}).reset_index()
df1

In [ ]:
df1 = df1.sort_values(by = "acqfromDt",ascending = False)
df1

In [ ]:
df1.head(50)